## Generate Morpheus Input Data

**Gregory Way, 2019**

Use this script to concatenate all of the cell painting data into one `.gct` file for input into morpheus.

In [1]:
import os
import pandas as pd

In [2]:
%load_ext rpy2.ipython

In [3]:
batch_id = "2019_04_16_Batch1"
backend_dir = os.path.join("..", "..", "backend", batch_id)

plate_dirs = [os.path.join(backend_dir, x) for x in os.listdir(backend_dir)]

In [4]:
# Build full cell painting dataset
df_list = []
for plate_dir in plate_dirs:
    plate_files = os.listdir(plate_dir)
    for plate_file in plate_files:
        if "normalized_variable_selected.csv" in plate_file:
            plate_file = os.path.join(plate_dir, plate_file)
            df = pd.read_csv(plate_file)
            print("reading {} with profile count: {}".format(plate_file, df.shape[0]))
            df_list.append(df)

reading ../../backend/2019_04_16_Batch1/BR00101080/BR00101080_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101081/BR00101081_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101075/BR00101075_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101079/BR00101079_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101077/BR00101077_normalized_variable_selected.csv with profile count: 56
reading ../../backend/2019_04_16_Batch1/BR00101083/BR00101083_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101082/BR00101082_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101076/BR00101076_normalized_variable_selected.csv with profile count: 54
reading ../../backend/2019_04_16_Batch1/BR00101078/BR001

In [5]:
# Combine into a single file
cp_df = pd.concat(df_list).reset_index(drop=True)
cp_df.Metadata_diff_day = cp_df.Metadata_diff_day.astype(str)

print(cp_df.shape)
cp_df.head()

(530, 549)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_cell_line,Metadata_patient,Metadata_FFA,Metadata_diff_day,Cells_AreaShape_EulerNumber,...,Nuclei_Texture_InfoMeas2_DNA_10_01,Nuclei_Texture_InfoMeas2_Mito_10_03,Nuclei_Texture_InverseDifferenceMoment_DNA_10_00,Nuclei_Texture_InverseDifferenceMoment_ER_10_02,Nuclei_Texture_InverseDifferenceMoment_Mito_10_02,Nuclei_Texture_InverseDifferenceMoment_Mito_20_02,Nuclei_Texture_SumAverage_DNA_20_03,Nuclei_Texture_SumVariance_AGP_20_01,Nuclei_Texture_SumVariance_ER_20_03,Nuclei_Texture_SumVariance_Mito_20_01
0,BR00101080,B02,BR00101080,BR00101080,B02,vc,PAC_261,0,3,-3.840565,...,-1.814082,0.039732,1.401266,-0.336906,-0.388075,2.894933,0.674111,0.915279,0.533406,0.954802
1,BR00101080,B03,BR00101080,BR00101080,B03,sc,PAC_246,0,3,-0.178843,...,-1.324510,-0.330499,1.344428,2.740432,0.738743,0.208088,-0.359262,-0.526420,-0.707300,-0.273817
2,BR00101080,B04,BR00101080,BR00101080,B04,vc,PAC_246,0,3,0.627837,...,-0.281201,0.867523,0.164746,-0.332073,-0.595390,-0.495383,-0.599370,-0.536304,-0.745026,-0.146896
3,BR00101080,B05,BR00101080,BR00101080,B05,sc,PAC_266,0,3,-0.134838,...,-0.480267,0.412462,0.433084,0.885338,-0.114261,-0.603112,-0.771305,-0.499049,-0.822919,-0.105638
4,BR00101080,B06,BR00101080,BR00101080,B06,vc,PAC_266,0,3,-3.069128,...,-2.483936,-1.712249,2.193710,1.653152,1.362144,6.744472,1.021106,0.937149,0.186470,0.288610


In [6]:
# Output combined file
file = os.path.join("data", "combined_normalized_variable_selected.tsv")
cp_df.to_csv(file, index=False, sep='\t')

# Collapse Data for Morpheus Heatmaps

In [7]:
# Extract out day 15 data
cp_15_df = cp_df.query("Metadata_diff_day in ['15', '15+iso']").reset_index(drop=True)
print(cp_15_df.shape)

cp_non15_df = cp_df.query("Metadata_diff_day not in ['15', '15+iso']").reset_index(drop=True)
print(cp_non15_df.shape)

(114, 549)
(416, 549)


In [8]:
replicate_cols = ["Metadata_Plate",
                  "Metadata_cell_line",
                  "Metadata_patient",
                  "Metadata_FFA",
                  "Metadata_diff_day"]

cp_15_collapsed_df = cp_15_df.groupby(replicate_cols).mean().reset_index()
cp_non15_collapsed_df = cp_non15_df.groupby(replicate_cols).mean().reset_index()

## Use `write_gct.R` to build the Morpheus Input

In [9]:
%%R -i cp_df -i batch_id -i backend_dir -i cp_15_df -i cp_non15_df -i cp_15_collapsed_df -i cp_non15_collapsed_df

library(dplyr)
library(magrittr)

file <- file.path("..", "cytominer_scripts", "write_gct.R")
source(file)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus.gct"))
channels <- NULL
create_row_annotations <- TRUE
feature_regex <- "^Nuclei_|^Cells_|^Cytoplasm_"

# Step 1: Output combined gct file
write_gct(x = cp_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

# Step 2: Output specific plate combinations
# (replicate collapsed and non replicate collapsed)
output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_day15.gct"))

write_gct(x = cp_15_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_nonday15.gct"))

write_gct(x = cp_non15_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_day15_collapsed.gct"))

write_gct(x = cp_15_collapsed_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_nonday15_collapsed.gct"))

write_gct(x = cp_non15_collapsed_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

# Step 3: Also generate and write individual gct files
plate_cols <- readr::cols(
    .default = readr::col_double(),
    Metadata_Plate = readr::col_character(),
    Metadata_Well = readr::col_character(),
    Metadata_Assay_Plate_Barcode = readr::col_character(),
    Metadata_Plate_Map_Name = readr::col_character(),
    Metadata_well_position = readr::col_character(),
    Metadata_cell_line = readr::col_character(),
    Metadata_patient = readr::col_character(),
    Metadata_FFA = readr::col_character(),
    Metadata_diff_day = readr::col_character()
)

all_plate_dirs <- list.files(backend_dir, full.names = TRUE)
for (plate_dir in all_plate_dirs) {
    plate_file <- list.files(plate_dir, full.names = FALSE, pattern = "normalized_variable_selected")[1]
    full_plate_file <- file.path(plate_dir, plate_file)

    df <- readr::read_csv(full_plate_file, col_types = plate_cols)
    
    output_file <- file.path("results", "morpheus",
                             paste0(tools::file_path_sans_ext(plate_file),
                                    "_", batch_id, "_morpheus.gct"))
    write_gct(x = df,
              path = output_file,
              channels = channels,
              create_row_annotations = create_row_annotations,
              feature_regex = feature_regex)
}

/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
